# Python for Data Analysis v2 | Notes_ Chapter_5 pandas 入门

本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [利用 Python 进行数据分析·第2版》第5章  pandas 入门](https://www.jianshu.com/p/161364dd0acf) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用 Python 进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境： Python  3.6 

---

## 第 5 章  pandas  入门

- pandas 是本书后续内容的首选库。它含有使数据清洗和分析工作变得更快更简单的数据结构和操作工具。 
- pandas 经常和其它工具一同使用，如数值计算工具 NumPy 和 SciPy ，分析库 statsmodels 和s cikit-learn，和数据可视化库 matplotlib。 
- pandas 是基于 NumPy 数组构建的，特别是基于数组的函数和不使用 for 循环的数据处理。
- 虽然 pandas 采用了大量的 NumPy 编码风格，但二者最大的不同是 pandas 是专门为处理表格和混杂数据设计的。而 NumPy 更适合处理统一的数值数组数据。


在本书后续部分中，我将使用下面这样的 pandas  引入约定：

```
In [1]: import pandas as pd

# 因此，只要你在代码中看到pd.，就得想到这是 pandas 。
# 因为 Series 和 DataFrame 用的次数非常多，所以将其引入本地命名空间中会更方便：

In [2]: from pandas import Series, DataFrame


```

### 5.1  pandas 的数据结构介绍

- 要使用 pandas ，你首先就得熟悉它的两个主要数据结构： Series 和 DataFrame 。
- 虽然它们并不能解决所有问题，但它们为大多数应用提供了一种可靠的、易于使用的基础。

###  Series 

Series 是一种类似于一维数组的对象，它由一组数据（各种 NumPy 数据类型）以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单的 Series ：

```
In [5]: obj = pd.Series([4, 8, -2, 3])

In [6]: obj
Out[6]:
0    4
1    8
2   -2
3    3
dtype: int64


```
Series 的字符串表现形式为：索引在左边，值在右边。由于我们没有为数据指定索引，于是会自动创建一个 0 到 N-1（ N 为数据的长度）的整数型索引。你可以通过 Series  的 values 和 index 属性获取其数组表示形式和索引对象：

```
In [7]: obj.values
Out[7]: array([ 4,  8, -2,  3], dtype=int64)

In [8]: obj.index  # like range(4)
Out[8]: RangeIndex(start=0, stop=4, step=1)

```

通常，我们希望所创建的 Series 带有一个可以对各个数据点进行标记的索引：

```
In [9]: obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', '
   ...: c'])

In [10]: obj2
Out[10]:
d    4
b    7
a   -5
c    3
dtype: int64

In [11]: obj2.index
Out[11]: Index(['d', 'b', 'a', 'c'], dtype='object')


```

与普通 NumPy 数组相比，你可以通过索引的方式选取 Series 中的单个或一组值：

```
In [12]: obj2['a']
Out[12]: -5

In [13]: obj2['b']
Out[13]: 7

In [14]: obj2[['c', 'd', 'a']]
Out[14]:
c    3
d    4
a   -5
dtype: int64


```
`['c', 'd', 'a']`是索引列表，即使它包含的是字符串而不是整数。

使用 NumPy 函数或类似 NumPy 的运算（如根据布尔型数组进行过滤、标量乘法、应用数学函数等）都会保留索引值的链接：

```
In [21]: obj2[obj2 > 0]
Out[21]:
d    4
b    7
c    3
dtype: int64

In [22]: obj2 * 2
Out[22]:
d     8
b    14
a   -10
c     6
dtype: int64

In [23]: import numpy as np

In [24]: np.exp(obj2)
Out[24]:
d      54.598150
b    1096.633158
a       0.006738
c      20.085537
dtype: float64


```
还可以将 Series 看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

```
In [25]: 'b' in obj2
Out[25]: True

In [26]: 'e' in obj2
Out[26]: False

# 如果数据被存放在一个 Python 字典中，也可以直接通过这个字典来创建 Series: 

In [27]: sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 1600
    ...: 0, 'Utah': 5000}

In [28]: obj3 = pd.Series(sdata)

In [29]: obj3
Out[29]:
Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64


```

如果只传入一个字典，则结果 Series 中的索引就是原字典的键（有序排列）。你可以传入排好序的字典的键以改变顺序：

```
In [30]: states = ['California', 'Ohio', 'Oregon', 'Texas']

In [31]: obj4 = pd.Series(sdata, index=states)

In [32]: obj4
Out[32]:
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

```
在这个例子中， sdata 中跟 states 索引相匹配的那 3 个值会被找出来并放到相应的位置上，但由于"California"所对应的 sdata 值找不到，所以其结果就为NaN（即“非数字”（not a number），在 pandas 中，它用于表示缺失或NA值）。因为‘Utah’不在 states 中，它被从结果中除去。

我将使用缺失（missing）或 NA 表示缺失数据。 pandas 的 isnull 和 notnull 函数可用于检测缺失数据：

```
In [33]: pd.isnull(obj4)
Out[33]:
California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [34]: pd.notnull(obj4)
Out[34]:
California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

```

Series 也有类似的实例方法：

```
In [35]: obj4.isnull()
Out[35]:
California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [36]: obj4.notnull()
Out[36]:
California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

```
我将在第7章详细讲解如何处理缺失数据。

对于许多应用而言， Series 最重要的一个功能是，它会根据运算的索引标签自动对齐数据：

```
In [37]: obj3
Out[37]:
Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

In [38]: obj4
Out[38]:
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [39]: obj3 + obj4
Out[39]:
California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

```

数据对齐功能将在后面详细讲解。如果你使用过数据库，你可以认为是类似join的操作。

Series 对象本身及其索引都有一个name属性，该属性跟 pandas 其他的关键功能关系非常密切：

```
In [43]: obj4.name = 'population'

In [44]: obj4.index.name= 'state'

In [45]: obj4
Out[45]:
state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

```

Series 的索引可以通过赋值的方式就地修改：

```
In [46]: obj
Out[46]:
0    4
1    8
2   -2
3    3
dtype: int64

In [47]: obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
    ...:

In [48]: obj
Out[48]:
Bob      4
Steve    8
Jeff    -2
Ryan     3
dtype: int64

```

###  DataFrame 

- DataFrame 是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）。
- DataFrame 既有行索引也有列索引，它可以被看做由 Series 组成的字典（共用同一个索引）。 
- DataFrame 中的数据是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。
- 笔记：虽然 DataFrame 是以二维结构保存数据的，但你仍然可以轻松地将其表示为更高维度的数据（层次化索引的表格型结构，这是 pandas 中许多高级数据处理功能的关键要素，我们会在第8章讨论这个问题）。


建 DataFrame 的办法有很多，最常用的一种是直接传入一个由等长列表或 NumPy 数组组成的字典：

```
In [50]: data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'N
    ...: evada', 'Nevada'],
    ...:         'year': [2000, 2001, 2002, 2001, 2002, 2003],
    ...:         'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
    ...:

In [51]: frame = pd.DataFrame(data)

# 结果 DataFrame 会自动加上索引（跟 Series 一样），且全部列会被有序排列：

In [52]: frame
Out[52]:
   pop   state  year
0  1.5    Ohio  2000
1  1.7    Ohio  2001
2  3.6    Ohio  2002
3  2.4  Nevada  2001
4  2.9  Nevada  2002
5  3.2  Nevada  2003

```

如果你使用的是 Jupyter notebook， pandas DataFrame 对象会以对浏览器友好的 HTML 表格的方式呈现。

对于特别大的 DataFrame ，head方法会选取前五行：


```
In [53]: frame.head()
Out[53]:
   pop   state  year
0  1.5    Ohio  2000
1  1.7    Ohio  2001
2  3.6    Ohio  2002
3  2.4  Nevada  2001
4  2.9  Nevada  2002


```
如果指定了列序列，则 DataFrame 的列就会按照指定顺序进行排列：

```

In [54]: pd.DataFrame(data, columns=['year', 'state', 'pop'])
Out[54]:
   year   state  pop
0  2000    Ohio  1.5
1  2001    Ohio  1.7
2  2002    Ohio  3.6
3  2001  Nevada  2.4
4  2002  Nevada  2.9
5  2003  Nevada  3.2

# 如果传入的列在数据中找不到，就会在结果中产生缺失值：

In [55]: frame2 = pd.DataFrame(data, columns=['year','state','p
    ...: op','debt'], index=['one', 'two', 'three','four', 'fiv
    ...: e', 'six'])

In [56]: frame2
Out[56]:
       year   state  pop debt
one    2000    Ohio  1.5  NaN
two    2001    Ohio  1.7  NaN
three  2002    Ohio  3.6  NaN
four   2001  Nevada  2.4  NaN
five   2002  Nevada  2.9  NaN
six    2003  Nevada  3.2  NaN

In [57]: frame2.columns
Out[57]: Index(['year', 'state', 'pop', 'debt'], dtype='object')

```

通过类似字典标记的方式或属性的方式，可以将 DataFrame 的列获取为一个 Series ：

```
In [58]: frame2['state']
Out[58]:
one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [59]: frame2
Out[59]:
       year   state  pop debt
one    2000    Ohio  1.5  NaN
two    2001    Ohio  1.7  NaN
three  2002    Ohio  3.6  NaN
four   2001  Nevada  2.4  NaN
five   2002  Nevada  2.9  NaN
six    2003  Nevada  3.2  NaN

In [60]: frame.year
Out[60]:
0    2000
1    2001
2    2002
3    2001
4    2002
5    2003
Name: year, dtype: int64

In [61]: frame2.year
Out[61]:
one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64


```
- 笔记：IPython 提供了类似属性的访问（即`frame2.year`）和 tab 补全。
- `frame2[column]`适用于任何列的名，但是`frame2.column`只有在列名是一个合理的 Python 变量名时才适用。

注意，返回的 Series 拥有原 DataFrame 相同的索引，且其 name 属性也已经被相应地设置好了。

行也可以通过位置或名称的方式进行获取，比如用 loc 属性（稍后将对此进行详细讲解）：

```
In [62]: frame2.loc['four']
Out[62]:
year       2001
state    Nevada
pop         2.4
debt        NaN
Name: four, dtype: object

```

列可以通过赋值的方式进行修改。例如，我们可以给那个空的"debt"列赋上一个标量值或一组值：

```


```

将列表或数组赋值给某个列时，其长度必须跟 DataFrame 的长度相匹配。如果赋值的是一个 Series ，就会精确匹配 DataFrame 的索引，所有的空位都将被填上缺失值：

```

```

为不存在的列赋值会创建出一个新列。关键字del用于删除列。

作为del的例子，我先添加一个新的布尔值的列，state是否为'Ohio'：

```

```
注意：不能用frame2.eastern创建新的列。

del方法可以用来删除这列：

- 注意：通过索引方式返回的列只是相应数据的视图而已，并不是副本。因此，对返回的 Series 所做的任何就地修改全都会反映到源 DataFrame 上。通过 Series 的copy方法即可指定复制列。

另一种常见的数据形式是嵌套字典：


```

```

如果嵌套字典传给 DataFrame ， pandas 就会被解释为：外层字典的键作为列，内层键则作为行索引：

```

```
你也可以使用类似 NumPy 数组的方法，对 DataFrame 进行转置（交换行和列）：

```

```
内层字典的键会被合并、排序以形成最终的索引。如果明确指定了索引，则不会这样：


```

```
由 Series 组成的字典差不多也是一样的用法：


```

```

表5-1列出了 DataFrame 构造函数所能接受的各种数据。


![](./images/5_1_ DataFrame .png)

如果设置了 DataFrame 的index和columns的name属性，则这些信息也会被显示出来：

```

```
跟 Series 一样，values属性也会以二维ndarray的形式返回 DataFrame 中的数据：

```

```
如果 DataFrame 各列的数据类型不同，则值数组的dtype就会选用能兼容所有列的数据类型：

```

```
### 索引对象

 pandas 的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建 Series 或 DataFrame 时，所用到的任何数组或其他序列的标签都会被转换成一个Index：

```

```
Index对象是不可变的，因此用户不能对其进行修改

```

```
不可变可以使Index对象在多个数据结构之间安全共享：

```

```

- 注意：虽然用户不需要经常使用Index的功能，但是因为一些操作会生成包含被索引化的数据，理解它们的工作原理是很重要的。

```

```

除了类似于数组，Index的功能也类似一个固定大小的集合：

```

```
与 Python 的集合不同， pandas 的Index可以包含重复的标签：


```

```
选择重复的标签，会显示所有的结果。

每个索引都有一些方法和属性，它们可用于设置逻辑并回答有关该索引所包含的数据的常见问题。表5-2列出了这些函数。

![](./images/5_2_index.png)

### 5.2 基本功能

本节中，我将介绍操作 Series 和 DataFrame 中的数据的基本手段。后续章节将更加深入地挖掘 pandas 在数据分析和处理方面的功能。本书不是 pandas 库的详尽文档，主要关注的是最重要的功能，那些不大常用的内容（也就是那些更深奥的内容）就交给你自己去摸索吧。

### 重新索引


 pandas 对象的一个重要方法是reindex，其作用是创建一个新对象，它的数据符合新的索引。看下面的例子：

```

```

用该 Series 的reindex将会根据新索引进行重排。如果某个索引值当前不存在，就引入缺失值：

```

```

对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。method选项即可达到此目的，例如，使用ffill可以实现前向值填充：

```

```
借助 DataFrame ，reindex可以修改（行）索引和列。只传递一个序列时，会重新索引结果的行：

```

```
列可以用columns关键字重新索引：

```

```
表5-3列出了reindex函数的各参数及说明。

![](./images/5_3_reindex.png)

### 丢弃指定轴上的项

丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象：

```

```
对于 DataFrame ，可以删除任意轴上的索引值。为了演示，先新建一个 DataFrame 例子：


```

```

用标签序列调用drop会从行标签（axis 0）删除值：

```

```

通过传递axis=1或axis='columns'可以删除列的值：

```

```


许多函数，如drop，会修改 Series 或 DataFrame 的大小或形状，可以就地修改对象，不会返回新的对象：

```

```

小心使用inplace，它会销毁所有被删除的数据。

### 索引、选取和过滤

 Series 索引（obj[...]）的工作方式类似于 NumPy 数组的索引，只不过 Series 的索引值不只是整数。下面是几个例子：



```

```
利用标签的切片运算与普通的 Python 切片运算不同，其末端是包含的：

```

```
用切片可以对 Series 的相应部分进行设置：

```

```
用一个值或序列对 DataFrame 进行索引其实就是获取一个或多个列：


```

```

这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取数据：

```

```
选取行的语法data[:2]十分方便。向[ ]传递单一的元素或列表，就可选择列。

另一种用法是通过布尔型 DataFrame （比如下面这个由标量比较运算得出的）进行索引

```

```


这使得 DataFrame 的语法与 NumPy 二维数组的语法很像。

### 用loc和iloc进行选取


对于 DataFrame 的行的标签索引，我引入了特殊的标签运算符loc和iloc。它们可以让你用类似 NumPy 的标记，使用轴标签（loc）或整数索引（iloc），从 DataFrame 选择行和列的子集。

作为一个初步示例，让我们通过标签选择一行和多列：



```

```
然后用iloc和整数进行选取：

```

```

这两个索引函数也适用于一个标签或多个标签的切片：

```

```
所以，在 pandas 中，有多个方法可以选取和重新组合数据。对于 DataFrame ，表5-4进行了总结。后面会看到，还有更多的方法进行层级化索引。


```

```

- 笔记：在一开始设计 pandas 时，我觉得用frame[:, col]选取列过于繁琐（也容易出错），因为列的选择是非常常见的操作。我做了些取舍，将花式索引的功能（标签和整数）放到了ix运算符中。在实践中，这会导致许多边缘情况，数据的轴标签是整数，所以 pandas 团队决定创造loc和iloc运算符分别处理严格基于标签和整数的索引。
ix运算符仍然可用，但并不推荐。

![](./images/5_4.png)

### 整数索引

处理整数索引的 pandas 对象常常难住新手，因为它与 Python 内置的列表和元组的索引语法不同。例如，你可能不认为下面的代码会出错：


```

```

这里， pandas 可以勉强进行整数索引，但是会导致小bug。我们有包含0,1,2的索引，但是引入用户想要的东西（基于标签或位置的索引）很难：


```

```
另外，对于非整数索引，不会产生歧义：


```

```

为了进行统一，如果轴索引含有整数，数据选取总会使用标签。为了更准确，请使用loc（标签）或iloc（整数）：

```

```
### 算术运算和数据对齐


 pandas 最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。对于有数据库经验的用户，这就像在索引标签上进行自动外连接。看一个简单的例子：

```

```

将它们相加就会产生：

```

```

自动的数据对齐操作在不重叠的索引处引入了NA值。缺失值会在算术运算过程中传播。

对于 DataFrame ，对齐操作会同时发生在行和列上：

```

```
把它们相加后将会返回一个新的 DataFrame ，其索引和列为原来那两个 DataFrame 的并集：

```

```

因为'c'和'e'列均不在两个 DataFrame 对象中，在结果中以缺省值呈现。行也是同样。

如果 DataFrame 对象相加，没有共用的列或行标签，结果都会是空：

```

```

### 在算术方法中填充值


在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值（比如0）：

```


```

将它们相加时，没有重叠的位置就会产生NA值：

```

```
表5-5列出了 Series 和 DataFrame 的算术方法。它们每个都有一个副本，以字母r开头，它会翻转参数。因此这两个语句是等价的：


```

```
![](./images/5_5.png)


```

```
与此类似，在对 Series 或 DataFrame 重新索引时，也可以指定一个填充值：
```

```

###  DataFrame 和 Series 之间的运算


跟不同维度的 NumPy 数组一样， DataFrame 和 Series 之间算术运算也是有明确规定的。先来看一个具有启发性的例子，计算一个二维数组与其某行之间的差：

```

```

当我们从arr减去arr[0]，每一行都会执行这个操作。这就叫做广播（broadcasting），附录A将对此进行详细讲解。 DataFrame 和 Series 之间的运算差不多也是如此：

```

```

默认情况下， DataFrame 和 Series 之间的算术运算会将 Series 的索引匹配到 DataFrame 的列，然后沿着行一直向下广播：


```

```


如果某个索引值在 DataFrame 的列或 Series 的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集：


```

```


如果你希望匹配行且在列上广播，则必须使用算术运算方法。例如：


```

```


传入的轴号就是希望匹配的轴。在本例中，我们的目的是匹配 DataFrame 的行索引（axis='index' or axis=0）并进行广播。

### 函数应用和映射

 NumPy 的ufuncs（元素级数组方法）也可用于操作 pandas 对象：


```

```
另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。 DataFrame 的apply方法即可实现此功能：

```

```


这里的函数f，计算了一个 Series 的最大值和最小值的差，在frame的每列都执行了一次。结果是一个 Series ，使用frame的列作为索引。

如果传递axis='columns'到apply，这个函数会在每行执行：


```

```

许多最为常见的数组统计功能都被实现成 DataFrame 的方法（如sum和mean），因此无需使用apply方法。

传递到apply的函数不是必须返回一个标量，还可以返回由多个值组成的 Series ：



```

```

元素级的 Python 函数也是可以用的。假如你想得到frame中各个浮点值的格式化字符串，使用applymap即可：



```

```

之所以叫做applymap，是因为 Series 有一个用于应用元素级函数的map方法

```

```

### 排序和排名

根据条件对数据集排序（sorting）也是一种重要的内置运算。要对行或列索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象



```

```


对于 DataFrame ，则可以根据任意一个轴上的索引进行排序：


```

```

数据默认是按升序排序的，但也可以降序排序：



```

```

若要按值对 Series 进行排序，可使用其sort_values方法：


```

```

在排序时，任何缺失值默认都会被放到 Series 的末尾：



```

```

当排序一个 DataFrame 时，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给sort_values的by选项即可达到该目的：



```

```


要根据多个列进行排序，传入名称的列表即可：


```

```

排名会从1开始一直到数组中有效数据的数量。接下来介绍 Series 和 DataFrame 的rank方法。默认情况下，rank是通过“为各组分配一个平均排名”的方式破坏平级关系的：


```

```

也可以根据值在原数据中出现的顺序给出排名：



```

```

这里，条目0和2没有使用平均排名6.5，它们被设成了6和7，因为数据中标签0位于标签2的前面。

你也可以按降序进行排名：



```

```

表5-6列出了所有用于破坏平级关系的method选项。 DataFrame 可以在行或列上计算排名：



```

```

![](./images/5_6.png)


### 带有重复标签的轴索引

直到目前为止，我所介绍的所有范例都有着唯一的轴标签（索引值）。虽然许多 pandas 函数（如reindex）都要求标签唯一，但这并不是强制性的。我们来看看下面这个简单的带有重复索引值的 Series ：

```

```

索引的is_unique属性可以告诉你它的值是否是唯一的：



```

```

对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多个值，则返回一个 Series ；而对应单个值的，则返回一个标量值：



```

```


这样会使代码变复杂，因为索引的输出类型会根据标签是否有重复发生变化。

对 DataFrame 的行进行索引时也是如此：


```

```

### 5.3 汇总和计算描述统计


 pandas 对象拥有一组常用的数学和统计方法。它们大部分都属于约简和汇总统计，用于从 Series 中提取单个值（如sum或mean）或从 DataFrame 的行或列中提取一个 Series 。跟对应的 NumPy 数组方法相比，它们都是基于没有缺失数据的假设而构建的。看一个简单的 DataFrame ：



```

```

调用 DataFrame 的sum方法将会返回一个含有列的和的 Series ：



```

```


传入axis='columns'或axis=1将会按行进行求和运算：


```

```


NA值会自动被排除，除非整个切片（这里指的是行或列）都是NA。通过skipna选项可以禁用该功能：


```

```

表5-7列出了这些约简方法的常用选项。

![](./images/5_7.png)


有些方法（如idxmin和idxmax）返回的是间接统计（比如达到最小值或最大值的索引）：

```

```

另一些方法则是累计型的：



```

```

还有一种方法，它既不是约简型也不是累计型。describe就是一个例子，它用于一次性产生多个汇总统计：



```

```

对于非数值型数据，describe会产生另外一种汇总统计：



```

```
表5-8列出了所有与描述统计相关的方法。

![](./images/5_8.png)

### 相关系数与协方差

有些汇总统计（如相关系数和协方差）是通过参数对计算出来的。我们来看几个 DataFrame ，它们的数据来自Yahoo!Finance的股票价格和成交量，使用的是 pandas -datareader包（可以用conda或pip安装）：


```

```
我使用 pandas _datareader模块下载了一些股票数据：

- 注意：此时Yahoo! Finance已经不存在了，因为2017年Yahoo!被Verizon收购了。参阅 pandas -datareader文档，可以学习最新的功能。

现在计算价格的百分数变化，时间序列的操作会在第11章介绍：

```

```

 Series 的 corr 方法用于计算两个 Series 中重叠的、非NA的、按索引对齐的值的相关系数。与此类似，cov用于计算协方差：

```

```
因为MSTF是一个合理的 Python 属性，我们还可以用更简洁的语法选择列：



```

```
另一方面， DataFrame 的 corr 和cov方法将以 DataFrame 的形式分别返回完整的相关系数或协方差矩阵：



```

```

利用 DataFrame 的 corr with方法，你可以计算其列或行跟另一个 Series 或 DataFrame 之间的相关系数。传入一个 Series 将会返回一个相关系数值 Series （针对各列进行计算）：


```

```

传入一个 DataFrame 则会计算按列名配对的相关系数。这里，我计算百分比变化与成交量的相关系数：

```

```

传入axis='columns'即可按行进行计算。无论如何，在计算相关系数之前，所有的数据项都会按标签对齐。

### 唯一值、值计数以及成员资格

还有一类方法可以从一维 Series 的值中抽取信息。看下面的例子：


```

```

第一个函数是unique，它可以得到 Series 中的唯一值数组：

```

```

返回的唯一值是未排序的，如果需要的话，可以对结果再次进行排序（uniques.sort()）。相似的，value_counts用于计算一个 Series 中各值出现的频率：

```

```
为了便于查看，结果 Series 是按值频率降序排列的。value_counts还是一个顶级 pandas 方法，可用于任何数组或序列：


```

```
isin用于判断矢量化集合的成员资格，可用于过滤 Series 中或 DataFrame 列中数据的子集：


```

```
与isin类似的是Index.get_indexer方法，它可以给你一个索引数组，从可能包含重复值的数组到另一个不同值的数组：

```

```

表5-9给出了这几个方法的一些参考信息。

![](./images/5_9.png)

有时，你可能希望得到 DataFrame 中多个相关列的一张柱状图。例如：

```

```

将 pandas .value_counts传给该 DataFrame 的apply函数，就会出现：


```

```


这里，结果中的行标签是所有列的唯一值。后面的频率值是每个列中这些值的相应计数。

### 5.4 总结

在下一章，我们将讨论用 pandas 读取（或加载）和写入数据集的工具。

之后，我们将更深入地研究使用 pandas 进行数据清洗、规整、分析和可视化工具。